# Упражнение 02 : Присоединяйтесь

## Разрешенные функции : импорт pandas как pd, импорт sqlite3

In [1]:
import pandas as pd 
import sqlite3

## создайте подключение к базе данных с помощью библиотеки sqlite3

In [4]:
conn=sqlite3.connect('../data/checking-logs.sqlite')
pd.read_sql("pragma table_info(pageviews)",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,datetime,TIMESTAMP,0,None,0


In [5]:
pd.read_sql("pragma table_info(checker)",conn)


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


## создайте новую таблицу в базе данных, объединив таблицы «pageviews» и «checker» с помощью всего одного запроса
### - в таблице должны быть следующие столбцы: uid, labname, first_commit_ts, first_view_ts
### - first_commit_ts — это просто новое название столбца timestamp в таблице проверки. Он показывает первую фиксацию для конкретной лаборатории и конкретного пользователя
### - first_view_ts — это первый визит пользователя в таблицу просмотров, отметка времени, когда пользователь посетил ленту новостей
### - status = ’готов’ по-прежнему должен быть фильтром
### - umTrials = 1 по-прежнему должно быть фильтром
### - Названия лабораторий должны быть из списка: «laba04», «laba04s», «laba05», «laba06», «laba06s», «project1»
### - таблица должна содержать только пользователей (идентификаторы с user_*), а не администраторов
### - first_commit_ts и first_view_ts должны быть преобразованы в тип datetime64[ns]

In [38]:

query = """
        create table if not exists datamart as
        select checker.uid, checker.labname, checker.timestamp as first_commit_ts,
        pageviews.datetime as first_view_ts from checker
        left join pageviews on pageviews.uid = checker.uid
        where checker.uid like 'user_%' and
        status = 'ready' and numTrials = 1 and
        labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
        and (pageviews.datetime = (select min(datetime) from pageviews where checker.uid = pageviews.uid) or pageviews.datetime is null)
        """

conn.execute(query)

df = pd.read_sql('select * from datamart', conn, parse_dates=['first_commit_ts', 'first_view_ts'])
df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


## используя методы Pandas, создайте два фрейма данных: test и control
### - у test должны быть пользователи, у которых есть значения в first_view_ts
### - у control должны быть пользователи, у которых в first_view_ts отсутствуют значения
### - замените отсутствующие значения в контрольной группе средним значением first_view_ts тестовых пользователей, мы будем использовать это значение для дальнейшего анализа
### - сохраните обе таблицы в базе данных, вы будете использовать их в следующих упражнениях

In [ ]:
test=df[df['first_view_ts'].notnull()]
control=df[df['first_view_ts'].isnull()]
control.loc[:, 'first_view_ts'] = test['first_view_ts'].mean()
test.info()
control.info()
test.to_sql("test", conn, index=False)
control.to_sql("control", conn, index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


81

In [43]:
pd.read_sql("pragma table_info(test)",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [44]:
pd.read_sql("pragma table_info(control)",conn)


,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## закройте соединение

In [46]:
conn.close()